In [ ]:
!pip install gymnasium[atari]
!pip install gymnasium matplotlib numpy
!pip install ale-py
!pip install gymnasium[accept-rom-license]
!pip install tensorflow


In [ ]:
from ale_py import ALEInterface
from ale_py.roms import Breakout
import gymnasium as gym
# from tensorflow import keras


In [ ]:
ale = ALEInterface()
ale.loadROM(Breakout)

In [ ]:
import random
env = gym.make("BreakoutNoFrameskip-v4", render_mode="human")
observation, info = env.reset()

next_frame, reward, terminated, truncated, info = env.step(1)
print('Reward Recieved = ' + str(reward))
print('Next state is a terminal state: ' + str(terminated))
print('info[ale.lives] tells us how many lives we have. Lives: ' + str(info['lives']))


for i in range(10000):
    a = random.sample([0,1,2,3] , 1)[0]
    f_p,r,d,t,info = env.step(a)
    print(r)
    env.render()
    if d == True:
        env.reset()
        